In [1]:
# !pip install openai

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print(WEAVIATE_URL[:10])
print(WEAVIATE_KEY[:10])
print(OPENAI_API_KEY[:10])

cwzujlbgtk
L1FUbzRPaU
sk-proj-iu


## Generate query from prompt

In [3]:
from openai import OpenAI

openai_client = OpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.openai.com/v1",
)

In [4]:
def generate_query_from_promt(prompt):
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            { "role": "system", "content": "Your job is to extract a query from the provided user prompt, the query will then be used to run a query in a vector database." },
            { 
                "role": "user",
                "content": f"Please give me a 2-3 word query that can be used to find relevant info to the following prompt - {prompt}"
            },
        ]
    )
    return response.choices[0].message.content

In [5]:
# Example of how to generate a query from a prompt
generate_query_from_promt("Where do the tallest penguins live?")

'"Tallest penguins location"'

## Connect to Weaviate

In [6]:
import weaviate
from weaviate.classes.init import Auth
# from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_KEY),

    headers = {
        "X-OpenAI-Api-Key": OPENAI_API_KEY
    },

    # additional_config=AdditionalConfig(
    #     timeout=Timeout(init=2, query=45, insert=120),  # Values in seconds
    # )
)

client.is_ready()

True

## Two-step RAG

In [7]:
def two_step_rag(user_prompt):
    # Step 1
    prompt = user_prompt + " Please only use the provided content with this prompt. Don't make things up."
    
    generated_query = generate_query_from_promt(prompt)
    print("=== Generated Query ===")
    print(f"Generated query: {generated_query}")

    # Step 2
    wiki = client.collections.get("Wiki")

    response = wiki.generate.near_text(
        query=generated_query,
        limit=3,
        grouped_task=prompt,
        grouped_properties=["text", "title"]
    )

    # Print results
    print("\n=== Generated Response ===")
    print(response.generative.text)

    print("\n=== Source ===")
    for item in response.objects:
        print(item.properties)

In [8]:
# two_step_rag("What wild animals do we know about?")
two_step_rag("Please provide an explanation at a highschool level. How do airplanes fly?")

=== Generated Query ===
Generated query: How airplanes fly

=== Generated Response ===
Airplanes fly mainly because of their wings, which are crucial for creating a force called lift. Lift works against gravity, allowing the airplane to rise off the ground. Here's how it happens:

When an airplane moves forward, air flows around its wings. The design of the wings helps push air downward. According to Bernoulli's principle, air moves faster over the top of the wing than underneath it. This creates a difference in pressure, with higher pressure on the bottom of the wing pushing the airplane upward.

Wings can also have flaps at the back, which can be extended to increase the wing's surface area. This increased size generates more lift but also more drag, which can help the plane fly more slowly without stalling. That's why flaps are commonly used during takeoff and landing.

In summary, airplanes utilize their wings to generate lift by manipulating airflow and air pressure, allowing them

In [9]:
two_step_rag("What are the pros and cons of automation using computer?")

=== Generated Query ===
Generated query: pros cons automation

=== Generated Response ===
**Pros of Automation Using Computers:**

1. **Efficiency and Mastery:** Automation can enhance efficiency as tasks that are initially performed with conscious effort (declarative knowledge) can become semi-automatic or unconscious (procedural knowledge) with mastery, leading to smoother and quicker performance.

2. **Reduction of Human Error:** By automating processes, the reliance on human factors decreases, potentially reducing errors that may occur due to fatigue or distraction.

3. **Consistency:** Automated systems can provide consistent outputs without the variability that human input typically introduces.

**Cons of Automation Using Computers:**

1. **Dependency on Specific Drivers:** Automation often requires specific device drivers to function correctly, which can make the setup process time-consuming and dependent on the operating system.

2. **Unconscious Biases:** Automation can someti

In [10]:
two_step_rag("How do CPUs work?")

=== Generated Query ===
Generated query: "How CPUs work"

=== Generated Response ===
CPUs, or Central Processing Units, are essential components of computers that carry out instructions from programs through a series of operations. Within the CPU, there are various units that perform specific tasks. One crucial part of the CPU is the execution unit. This execution unit is responsible for performing operations or calculations by executing the instructions provided to it.

The execution unit might include its own internal control sequence unit to oversee the execution of instructions, and it also contains registers for temporarily holding data. Additionally, it may have specialized components, such as a sub-ALU (Arithmetic Logic Unit) or FPU (Floating Point Unit), which help in carrying out more complex mathematical operations or specific types of calculations efficiently.

In summary, a CPU works by utilizing its execution units to perform computations and process instructions, orchestr

## Close the client

In [11]:
client.close()